# LSTM for text prediction

## Importing the libraries

In [279]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import os
from torch.utils.tensorboard import SummaryWriter
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
# import torchtext.transforms as T


C:\Users\nadav\anaconda3\envs\ZarembaLSTMS\lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
C:\Users\nadav\anaconda3\envs\ZarembaLSTMS\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
C:\Users\nadav\anaconda3\envs\ZarembaLSTMS\lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORT

# Tensorboard
tensorboard is a visualization tool that can be used to visualize the training process of a deep learning model. The torch.utils.tensorboard.SummaryWriter class is used to write the logs to the tensorboard. The SummaryWriter class takes the log directory as input. The logs are written to the log directory in the form of event files. The event files can be visualized using the tensorboard web interface.

In [182]:
#%tensorboard --logdir runs
log_dir = os.path.join(os.getcwd(), "runs")
os.makedirs(log_dir, exist_ok=True)



# Data Preprocessing
we will train the LSTM model on the Penn  Treebank dataset. The Penn Treebank dataset is a dataset of cleaned and annotated English text. The data is split into training, validation, and testing sets.

## load train, test, and validation data

In [449]:
train_data_raw = open('data/ptb.train.txt', 'r').read()
test_data_raw = open('data/ptb.test.txt', 'r').read()
valid_data_raw = open('data/ptb.valid.txt', 'r').read()
data =  train_data_raw + ' ' + test_data_raw + ' ' + valid_data_raw



### sample from the data
 let's see what are the most common words in the data

In [450]:
from collections import Counter

leaderboard = Counter(data.split()).most_common(10)
i = 0
for word, freq in leaderboard:
    i+=1
    print(f'{i}.{word}: appears {freq} times')

1.the: appears 59421 times
2.<unk>: appears 53299 times
3.N: appears 37607 times
4.of: appears 28427 times
5.to: appears 27430 times
6.a: appears 24755 times
7.in: appears 21032 times
8.and: appears 20404 times
9.'s: appears 11555 times
10.for: appears 10436 times


## Tokenizing the data

In [451]:
# Tokenize the data
def tokenize(text):
    return text.replace('\n', '<eos>').split()


### create a vocabulary of words


In [456]:
def build_vocab(text):
    tokens = tokenize(text)
    counter = Counter(tokens)
    vocab = sorted(counter, key=counter.get, reverse=True)
    vocab = {word: i for i, word in enumerate(vocab, 1)}

    return vocab


vocab = build_vocab(data)
vocab_size = len(vocab) + 1



print(f'vocab size: {vocab_size}')

vocab size: 10001


### decode and encode the words
 let's create a function that converts a word to token index and vice versa: the function stoi converts a word to a token index and the function itos converts a token index to a word

In [457]:

# decode the token i to a word S
def itos(i):
    return list(vocab.keys())[i-1]

# encode the word S to a token index i
def stoi(s):
    return vocab[s] if s in vocab else vocab['<unk>']

In [459]:
print(vocab)

print(stoi('year'))
print(itos(42))



{'the': 1, '<unk>': 2, '<eos>': 3, 'N': 4, 'of': 5, 'to': 6, 'a': 7, 'in': 8, 'and': 9, "'s": 10, 'for': 11, 'that': 12, '$': 13, 'is': 14, 'it': 15, 'said': 16, 'on': 17, 'at': 18, 'by': 19, 'as': 20, 'from': 21, 'with': 22, 'million': 23, 'mr.': 24, 'was': 25, 'be': 26, 'its': 27, 'are': 28, 'he': 29, 'but': 30, 'has': 31, "n't": 32, 'an': 33, 'have': 34, 'will': 35, 'new': 36, 'or': 37, 'company': 38, 'they': 39, 'this': 40, 'which': 41, 'year': 42, 'would': 43, 'about': 44, 'market': 45, 'says': 46, 'more': 47, 'were': 48, 'had': 49, 'billion': 50, 'their': 51, 'his': 52, 'up': 53, 'u.s.': 54, 'one': 55, 'than': 56, 'stock': 57, 'been': 58, 'some': 59, 'who': 60, 'also': 61, 'other': 62, 'share': 63, 'not': 64, 'we': 65, 'corp.': 66, 'when': 67, 'last': 68, 'if': 69, 'i': 70, 'all': 71, 'shares': 72, 'president': 73, 'years': 74, 'trading': 75, 'first': 76, 'two': 77, 'after': 78, 'inc.': 79, 'because': 80, 'could': 81, 'sales': 82, '&': 83, 'out': 84, 'there': 85, 'do': 86, 'only'

In [468]:
train_data = [stoi(word) for word in train_data_raw.split()]
valid_data = [stoi(word) for word in valid_data_raw.split()]
test_data = [stoi(word) for word in test_data_raw.split()]

## build a dataset and dataloader

In [469]:
batch_size = 20
seq_length = 32

In [470]:
class PTBDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) // self.seq_length

    def __getitem__(self, idx):
        x = self.data[idx * self.seq_length: (idx + 1) * self.seq_length]
        y = self.data[idx * self.seq_length + 1: (idx + 1) * self.seq_length + 1]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)


# Create datasets
train_dataset = PTBDataset(train_data, seq_length)
valid_dataset = PTBDataset(valid_data, seq_length)
test_dataset = PTBDataset(test_data, seq_length)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

test_data_iter = iter(test_loader)
train_data_iter = iter(train_loader)
valid_data_iter = iter(valid_loader)

### display a batch of data
note that y is exactly x shifted by one position. meaning that $y_i = x_{i+1} = LSTM(x[0:i-1])$

In [471]:
x, y = next(train_data_iter)
print(f'x: {x.size()}, y: {y.size()}')
print(f'x:{" ".join([itos(i) for i in x[0]])}')
print(f'y:{" ".join([itos(i) for i in y[0]])}')

x: torch.Size([20, 32]), y: torch.Size([20, 32])
x:in legal briefs the government 's startling allegations filed only days before the scheduled start of a criminal <unk> trial against ge in philadelphia federal district court challenge the <unk> and <unk>
y:legal briefs the government 's startling allegations filed only days before the scheduled start of a criminal <unk> trial against ge in philadelphia federal district court challenge the <unk> and <unk> of


# Lstm model Architecture

In [498]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTM_Cell(nn.Module):
    def __init__(self,
                 input_size,
                 hidden_size=200,
                 num_layers=1,
                 dropout=0,
                 ):
        """
        :param input_size:
        :param hidden_size:
        :param num_layers:
        :param num_classes:
        """
        super(LSTM_Cell, self).__init__()

        # size of the hidden state
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(dropout)


        # LSTM gates
        # Forget gate
        self.f_gate = nn.Sequential(
            nn.Linear(input_size + hidden_size, hidden_size),
            nn.Sigmoid() )

        # Candidate gate(input modulation in the original paper)
        self.g_gate = nn.Sequential(
            nn.Linear(input_size + hidden_size, hidden_size),
            nn.Tanh())

        # Input gate
        self.i_gate = nn.Sequential(
            nn.Linear(input_size + hidden_size, hidden_size),
            nn.Sigmoid())

        # Output gate
        self.o_gate = nn.Sequential(
            nn.Linear(input_size + hidden_size,hidden_size),
            nn.Sigmoid())
        self.tanh = nn.Tanh()

    def forward(self,x,h,c):
        """
        :param x: input tensor
        :param h: previous hidden state
        :param c: previous cell state
        :return: (h,c) tuple of new cell state and new hidden state
        """
        # Concatenate input and hidden state
        x_h = torch.cat((x,h),dim=1)

        # Forget
        f = self.f_gate(x_h)
        g = self.g_gate(x_h)
        i = self.i_gate(x_h)
        o = self.o_gate(x_h)

        # update c
        c = c * f + (g*i)
        # THEN, update h
        h = self.tanh(c) * o

        # apply dropout to the hidden
        h = self.dropout(h)

        return h,c

class LSTM(nn.Module):
    def __init__(self,
                 input_size=200,
                 hidden_size=32,
                 num_layers=1,
                 vocab_size=vocab_size,
                 dropout=0):
        """
        :param input_size:
        :param hidden_size:
        :param num_layers:
        """
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, input_size)

        self.LSTM_Cell = LSTM_Cell(input_size,
                                   hidden_size,
                                   num_layers,
                                   dropout=dropout)
        self.hidden_size = hidden_size
        self.fc = nn.Linear(hidden_size, vocab_size)


    def forward(self,x):
            """
            :param x: input tensor
            :return: (c,h) tuple of new cell state and new hidden state
            """
            batch_size = x.size(0)
            h = [torch.zeros(batch_size,self.hidden_size)]
            outputs = []
            # store the initial hidden and cell states
            ht =torch.zeros(batch_size, self.hidden_size) # store the hidden states (output)
            c = torch.zeros(batch_size,self.hidden_size)

            # store the hidden states (output)

            for i in range(x.size(1)):
                xi = self.embedding(x[:,i])

                ht,c = self.LSTM_Cell(xi,ht,c)
                h.append(ht)

                outputs.append(self.fc(h[-1]))
            # (seq_length, batch_size, vocab_size) => (batch_size, seq_length, vocab_size)

            return torch.stack(outputs, dim=1)


In [499]:

tb_writer = SummaryWriter('runs/LSTM')
lstm_cell = LSTM_Cell(32, 32, 1)
# visualize the model in our tensorboard summary
tb_writer.add_graph(lstm_cell, [torch.rand(32),torch.rand(32),torch.rand(32)])
tb_writer.flush()

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [500]:
def evaluate(model,
             val_loader,
             criterion = nn.CrossEntropyLoss(),
             seq_length = 32):
    """
    evaluate the model on the validation set
    :param model: model to evaluate
    :param val_loader: validation dataset loader
    :return: (accuracy, loss)
    """
    model.eval()
    running_loss = 0.0
    running_acc = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(val_loader, 0):
            inputs, targets = data

            outputs = model(inputs)

            outputs = outputs.view(-1, vocab_size)
            targets = targets.view(-1)

            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            running_acc += (predicted == targets).sum().item()
            total += targets.size(0)
        avg_loss = running_loss/ len(val_loader)
        perplexity = np.exp(avg_loss)
    return perplexity, loss

# Example

In [476]:
x,y = next(train_data_iter)
print(x.size())
lstm = LSTM(200, 200, 1)

evaluate(lstm, valid_loader)



torch.Size([20, 32])


(10070.26445123763, tensor(9.2142))

In [501]:
from datetime import datetime
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001)
epochs = 10

def train(model,
          train_loader=train_loader,
          val_loader=valid_loader,
          criterion = criterion,
          optimizer = optimizer,
          epochs=epochs,
          session = None,
          ):
    #create /models directory IF it does not exist
    if not os.path.exists('/models'):
        # Create the directory
        os.makedirs('/models')

    # track with tensorboard
    session = session or 'LSTM'+datetime.now().strftime('%m-%d-%H-%M')
    #tb_writer = SummaryWriter(f'runs/{session}')
    run_dir = f'{log_dir}/{session}'
    print(run_dir)
    tb_writer = SummaryWriter(run_dir)
    tb_writer.flush()

    val_loss_min = np.Inf
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        running_corrects = 0
        train_total = 0

        # set the model to train mode
        model.train(True)

        for i, data in enumerate(train_loader, 0):

            # get the input image and labels
            inputs, labels = data

            # start with zero gradients
            optimizer.zero_grad()
            # forward pass
            outputs = model(inputs)

            # reshape the outputs and labels (batch_size , seq_length, vocab_size)=>(batch_size * seq_length, vocab_size)
            outputs = outputs.view(-1, vocab_size)
            labels = labels.view(-1)

            # calculate the loss
            loss = criterion(outputs, labels)
            loss.backward()
            # update the weights
            optimizer.step()
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)

            running_corrects += (preds == labels).sum().item()
            train_total += labels.size(0)

            if i % 100 == 99:
                # print gradient statistics
                for name, param in model.named_parameters():
                    tb_writer.add_histogram(name, param.grad, epoch * len(train_loader) +  i)
                # loss of current batch
                avg_train_loss = running_loss / 100
                tb_writer.add_scalar('training loss', avg_train_loss, epoch * len(train_loader) +  i)

                #print(f'[{epoch + 1}, {i + 1}] loss: {avg_train_loss}')
                running_loss = 0.0
        """-----------------
        per epoch evaluation
        -----------------"""
        # set the model to evaluation mode
        model.eval()

        # TODO convert accuracy to perplexity
        # TODO: maybe move perplexity tensorboard log to happen more often
        train_perplexity = np.exp(avg_train_loss)
        # validation
        val_perplexity, val_loss = evaluate(model, val_loader)

        #val_accuracy = (val_preds == val_labels).sum().item() / len(val_loader)
        tb_writer.add_scalars('train vs val loss', {'train': avg_train_loss, 'val': val_loss}, epoch)
        tb_writer.add_scalars('train vs val perplexity', {'train': train_perplexity, 'val': val_perplexity}, epoch)
        print(f'at epoch {epoch}: \nvalidation loss: {val_loss} \ntraining loss:   {avg_train_loss} ')
        tb_writer.add_scalar('validation loss', val_loss, epoch)
        if val_loss <= val_loss_min:
            print('validation loss decreased({:.6f} -->{:.6f}). Saving Model ...'.format(val_loss_min, val_loss))
            torch.save(model, f'./models/ {session}.pt')
            val_loss_min = val_loss
    print('Finished Training')

In [494]:

lstm = LSTM(input_size=200,
            hidden_size=200,
            num_layers=1,
            vocab_size=vocab_size,
            dropout=0.5)
train(lstm, session='dropout LSTM')

C:\Users\nadav\DataspellProjects\ZarembaLSTMS\runs/vanilla LSTM
at epoch 0: 
validation loss: 5.488518238067627 
training loss:   0.3099268746186953 
validation loss decreased(inf -->5.488518). Saving Model ...
at epoch 1: 
validation loss: 5.491287708282471 
training loss:   0.2977691882792874 
validation loss decreased(inf -->5.491288). Saving Model ...
at epoch 2: 
validation loss: 5.517038822174072 
training loss:   0.28723508398912756 
validation loss decreased(inf -->5.517039). Saving Model ...
at epoch 3: 
validation loss: 5.564173221588135 
training loss:   0.28419930176848374 
validation loss decreased(inf -->5.564173). Saving Model ...
at epoch 4: 
validation loss: 5.612053394317627 
training loss:   0.2745754407650288 
validation loss decreased(inf -->5.612053). Saving Model ...
at epoch 5: 
validation loss: 5.665568828582764 
training loss:   0.2711103254715659 
validation loss decreased(inf -->5.665569). Saving Model ...
at epoch 6: 
validation loss: 5.737884521484375 
tra

In [495]:
# load a random sample from the test data
x, y = next(test_data_iter)

out = lstm(x)[-1]
print("====================================")
print(x.size())
print(itos(42))
print(itos(x[0][0].item()))
print("input: ", " ".join([itos(i) for i in x[0]]))
print("output: ", " ".join([itos(i) for i in out.argmax(dim=1)]))

print("===========")
for i in range(5,15):

    print( " ".join([itos(i) for i in x[0]][:i]),itos(out.argmax(dim=1)[i].item()))

torch.Size([20, 32])
year
negative
input:  negative <unk> to its own balance sheet mr. <unk> said the company is putting up only N N of the capital although it is responsible for providing management planning and processing services
output:  ranges for the <unk> <unk> the that to the heat <unk> be be a in the <unk> of banks are they on the will been in the far trading be the the
negative <unk> to its own the
negative <unk> to its own balance that
negative <unk> to its own balance sheet to
negative <unk> to its own balance sheet mr. the
negative <unk> to its own balance sheet mr. <unk> heat
negative <unk> to its own balance sheet mr. <unk> said <unk>
negative <unk> to its own balance sheet mr. <unk> said the be
negative <unk> to its own balance sheet mr. <unk> said the company be
negative <unk> to its own balance sheet mr. <unk> said the company is a
negative <unk> to its own balance sheet mr. <unk> said the company is putting in
